# Sprawozdanie z **Obliczeń ewolucyjnych**
## Zadanie 1 - Algorytm *Particle Swarm Optimization*

# Konfiguracja
Przy pomocy klasy Config można wybrać zastosowany algorytm oraz określić jego parametry:    
- w - współczynnik zmęczenia.   
- c1 - współczynnik przyspieszenia w kierunku własnego minimum cząstki
- c2 - współczynnik przyspieszenia w kierunku globalnego minimum cząstek
- iterations - ilość iteracji algorytmu  
- target_error - najniższy stopień błędu - po osiągnięciu tego wyniku algorytm zatrzymuje się  
- n_particles - ilość cząstek  
- arguments_dimensions - liczba wymiarów, w których rozkładane są cząsteczki   
- d_min - minimalna wartość współrzędnej dla każdego z wymiarów   
- d_max - maksymalna wartość współrzędnej dla każdego z wymiarów   
- fitness - wybór funkcji dostosowującej położenie cząstek         
 Dla celów badawczych, w dalszej części zaprezentowane są różne zestawy wartości konfiguracyjnych.

In [2]:
!pip install deap k3d > /dev/null
%matplotlib inline
from pathlib import Path
import json

class Config(object):
    def __init__(self, algorithmType, w, c1, c2, custom_c2, iterations, target_error, n_particles, arguments_dimensions, d_min, d_max, fitness):
        self.algorithmType = algorithmType
        self.w = w
        self.c1 = c1
        self.c2 = c2
        self.custom_c2 = custom_c2
        self.iterations = iterations
        self.target_error = target_error
        self.n_particles = n_particles
        self.arguments_dimensions = arguments_dimensions
        self.d_min = d_min
        self.d_max = d_max
        self.d_max = d_max
        self.fitness = fitness

def as_config(dct):
    return Config(
        dct['algorithmType'],
        dct['w'],
        dct['c1'],
        dct['c2'],
        dct['custom_c2'] if dct.get('custom_c2') else False,
        dct['iterations'], 
        dct['target_error'], 
        dct['n_particles'],
        dct['arguments_dimensions'],
        dct['d_min'], 
        dct['d_max'],
        dct['fitness']
        )

# Algorytm genetyczny z użyciem "deap"
W celu porównania samodzielnie zaimplementowanego algorytmu PSO do algorytmów genetycznych, poniżej zaimplementowano funkcję zwracające wyniki działania tego algorytmu z wykorzystaniem biblioteki "deap".  

<div style="text-align: justify;">Algorytm genetyczny (ang. <i>genetic algorithm</i>) -
jest jedną z ewolucyjnych metod
optymalizacji. Zalicza się go do klasy
algorytmów heurystycznych.
Przeszukiwanie możliwych rozwiązań w
celu znalezienia rozwiązania najlepszego
lub potencjalnie najlepszego odbywa się za
pomocą mechanizmów ewolucji oraz
doboru naturalnego.
</div>
<div> Koncepcyjnie algorytm składa się z następujących kroków:
<ol>
    <li>Inicjujemy (najczęściej w sposób losowy) pewną
początkową populację osobników</li>
    <li>Poddajemy każdego z nich ocenie</li>
    <li>Z populacji wybieramy osobniki najlepiej
do tego przystosowane</li>
    <li>Za pomocą operacji genetycznych
(krzyżowanie oraz mutacja) tworzymy
nowe pokolenie</li>
    <li>Powrót do punktu 2</li>
</ol>
</div>
<div style="text-align: justify;"> Potencjalne rozwiązania traktowane są jako
osobniki populacji. Algorytm symuluje proces
naturalnej selekcji poprzez ocenę przystosowania
poszczególnych osobników, eliminację osobników
słabszych i krzyżowanie ze sobą osobników
najsilniejszych. Wynikiem działania algorytmu
genetycznego jest populacja najlepiej
przystosowanych osobników, wśród których
może znajdować się najlepsze rozwiązanie.
Jednocześnie najlepiej przystosowane osobniki nie
muszą leżeć blisko siebie w przestrzeni rozwiązań.
</div><p></br></p>
<div>
    Ustawiono następujące wartości parametrów konfiguracyjnych dla algorytmu genetycznego:
</div>
<ul>
    <li>Prawdopodobieństwo mutacji (<i>mutpb</i>) - 0.2</li>
    <li>Prawdopodobieństwo krzyżowania (<i>cxpb</i>, ang. crossover probability) - odstetek w jakim nowa populacja składa się z krzyżówek, a w jakim stopniu z niezmienionych osobników</li>
    <li>Funkcja mutacji (<i>mutate</i>) - "mutFlipBit" oznacza funkcję odwracającą wartość poszczególnych atrybutów z określonym prawdopodobieństwem  </li>
    <li>Metoda selekcji (<i>select</i>) - turniejowa, rozmiar turnieji (<i>tournsize</i>) wynosi 3</li>
    <li>Ilość generacji (<i>ngen</i>) - kontrolowana wartością "iterations" z uprzednio opisanej konfiguracji</li>
    <li>Typ krzyżowania (<i>mate</i>) - "cxTwoPointCopy" oznacza krzyżowanie typu <i>two point crossover</i>, w którym losuje się dwa punkty dzielące chromosomy rodziców na trzy części. Dzieci powstają poprzez zamianę miejscami środkowego odcinka chromosomów rodziców.
</ul>

In [15]:
import random
import numpy
import inspect
from deap import algorithms, base, creator, tools

# translates into tuple
def gaFitness(individual):
    return fitness(individual),

def cxTwoPointCopy(ind1, ind2):
    size = len(ind1)
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] \
        = ind2[cxpoint1:cxpoint2].copy(), ind1[cxpoint1:cxpoint2].copy()
        
    return ind1, ind2

def genetic():
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", numpy.ndarray, fitness=creator.FitnessMin)

    toolbox = base.Toolbox()
    toolbox.register("attr_bool", lambda: (cfg.d_max-cfg.d_min)*random.random() + cfg.d_min)
    toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=cfg.arguments_dimensions)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    toolbox.register("evaluate", gaFitness)
    toolbox.register("mate", cxTwoPointCopy)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)
    pop = toolbox.population(n=cfg.n_particles)
    hof = tools.HallOfFame(1, similar=numpy.array_equal)
    
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    return algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=cfg.iterations, stats=stats, halloffame=hof, verbose=False)

# Klasy algorytmu PSO
Poniżej przedstawione są klasy repezentujące "rój".
### Particle
Klasa reprezentacja jedną cząsteczkę w przestrzeni poszukiwań:  
- position - pozycja w przestrzeni zdefiniowanej w pliku konfiguracyjnym. 
- pbest_position - najlepsza pozycja cząsteczki
- pbest_value - najlepsza osiągnięta przez tę czastęczkę wartość
- velocity - wektor, który przechowuje informacje nt. tzw prędkości cząsteczki.  
### Space
Klasa reprezentująca przestrzeń poszukiwań, w której poruszają się cząsteczki
- particles[] - tablica cząsteczek
- gbest_position - globalnie najlepsza pozycja cząsteczki - inicjowana losowo 
- gbest_value - najlepsza wartość funkcji przystosowania dowolnej z cząsteczek

In [4]:
import random
import numpy as np

class Particle():
    def __init__(self):
        self.position = (cfg.d_max - cfg.d_min) * np.random.random(cfg.arguments_dimensions) + cfg.d_min
        self.pbest_position = self.position
        self.pbest_value = float('inf')
        self.velocity = np.zeros(cfg.arguments_dimensions)
    
    def move(self):
        self.position = self.position + self.velocity
        for i in range(len(self.position)):
            if self.position[i] > cfg.d_max:
                self.position[i] = cfg.d_max
            if self.position[i] < cfg.d_min:
                self.position[i] = cfg.d_min

    def __str__(self):
        print(f'My position is {self.position}, my pbest is {self.pbest_position}')

class Space():
    def __init__(self):
        self.particles = []
        self.gbest_value = float('inf')
        self.gbest_position = np.array([x * np.random.random_sample()*cfg.d_max for x in range(cfg.arguments_dimensions)])

    def print_particles(self):
        for particle in self.particles:
            particle.__str__()

    def set_pbest(self):
        for particle in self.particles:
            fitness_cadidate:float = fitness(particle.position)
            if(particle.pbest_value > fitness_cadidate):
                particle.pbest_value = fitness_cadidate
                particle.pbest_position = particle.position

    def set_gbest(self):
        for particle in self.particles:
            best_fitness_cadidate = fitness(particle.position)
            if(self.gbest_value > best_fitness_cadidate):
                self.gbest_value = best_fitness_cadidate
                self.gbest_position = particle.position

    def move_particles(self):
        for particle in self.particles:
            new_velocity = (cfg.w*particle.velocity) + (cfg.c1*np.random.random_sample()) * (particle.pbest_position - particle.position) + \
                            (np.random.random_sample()*cfg.c2) * (self.gbest_position - particle.position)
            particle.velocity = new_velocity
            particle.move()


# Wykresy
Wykorzystano bibliotekę k3d w celu przedstawienia trójwymiarowych wykresów "roju", tam gdzie było to możliwe.

In [5]:
from numpy import exp,array,float32
import k3d
from uuid import uuid4

def plot_particles(particles):
    if len(particles) == 0:
        print("No particles to visualize !")
        return
    if len(particles[0].position) != 2:
#         print("I can only visualize three dimensional function !")
        return
    x = list(map(lambda p: (p.position[0], p.position[1], fitness(p.position)), particles))
    plot = k3d.plot(name=str(uuid4()))
    plt_points = k3d.points(positions=x, point_size=0.2)
    plot += plt_points
    plt_points.shader='3d'
    plot.display()

<h2>Funkcje przystosowania</h2>

<div>Poniższa sekcja przestawia zaimplementowane funkcje dostosowania cząsteczek. Wybranie funkcji następuje przy pomocy odpowiedniego parametru w konfiguracji.
</div>
<p> </p>
<div>
<p>Sincos</p>
<img src="pics/sincos.png" width="300">
Sincos2
<img src="pics/sincos2.png" width="300">
Sphere
    <img src="pics/sphere.png" width="200">
F2
    <img src="pics/f2.png" width="200">
Griewank
    <img src="pics/griewank.png" width="300">
Rosenbrock
    <img src="pics/rosen.png" width="300">
Zakharov
    <img src="pics/zakharov.png" width="300">
</div>

In [6]:
from numpy import sum, asarray_chkfinite, arange, prod, sqrt, cos, sin
from functools import reduce
def fitness(position, fr=4000):
    if cfg.fitness == "sincos":
        return sin(position[0]) + cos(position[1])
    if cfg.fitness == "sincos2":
        return sin(position[0]) + cos(position[1]) + position[0] + position[1]
    elif cfg.fitness == "sphere":
        return reduce(lambda p,n: p + n**2, position)
    elif cfg.fitness == "rosen":
        return sum(100.0*(position[1:]-position[:-1]**2.0)**2.0 + (position[:-1]-1)**2.0)
    elif cfg.fitness == "griewank":
        n = len(position)
        j = arange( 1., n+1 )
        s = sum( position**2 )
        p = prod( cos( position / sqrt(j) ))
        return s/fr - p + 1
    elif cfg.fitness == "zakharov":
        x = np.asarray_chkfinite(position)
        n = len(x)
        j = np.arange( 1., n+1 )
        s2 = sum( j * x ) / 2
        return sum( x**2 ) + s2**2 + s2**4
    elif cfg.fitness == "f2":
        s = 0
        for x in range(1, len(position)):
            s = s + (position[x]-x)**2
        return s
    else:
        print("Unknown fitenss function")
        raise

# Pętla aplikacji
Pętla programowa odpowiedzialna za wywoływanie logiki wybranego algorytmu PSO lub GA.

In [7]:
import numpy as np
class App():
    def run(self, config_prop="arguments_dimensions"):
        if(cfg.algorithmType == "pso"):
            c2_begin = cfg.c2
            search_space = Space()
            particles_vector = [Particle() for _ in range(cfg.n_particles)]
            search_space.particles = particles_vector
            plot_particles(search_space.particles)            
#             search_space.print_particles()
            iteration = 0
            while iteration < cfg.iterations:
                if cfg.custom_c2 == True:
                    cfg.c2 = c2_begin * ((cfg.iterations*0.9 - iteration)/cfg.iterations)
                last_gbest = search_space.gbest_value
                search_space.set_pbest()
                search_space.set_gbest()
                search_space.move_particles()
                iteration += 1
                if abs(last_gbest - search_space.gbest_value) < cfg.target_error:
                    break
            print(f"Fitness function value: {fitness(search_space.gbest_position)} achieved for {config_prop} = {getattr(cfg, config_prop)}.\nThe best solution is in: {search_space.gbest_position}.\nAchieved in {iteration} iterations")
#             plot_particles(search_space.particles)
#             search_space.print_particles()
        elif(cfg.algorithmType == "ga"):
            return genetic()[-1][-1]

        else:
            print(f"Unknown algorithm type: {cfg.algorithmType}")

### Ziarno losowości

In [8]:
from numpy import random as nrand
nrand.seed(0)
from random import seed
seed(0)

# Badania
Przeprowadzono eksperyment uruchomień algorytmu dla różnych zestawów parametrów konfiguracyjnych. Poniżej przedstawiono poszczególne przykłady wraz z wynikami w postaci wykresów 3D lub najlepszego osiągniętego wyniku.

### Początkowa konfiguracja

In [9]:
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "griewank",
    "w": 0.5,
    "c1": 0.9,
    "c2": 0.7,
    "custom_c2": false,
    "target_error": 0,
    "iterations": 2000,
    "n_particles": 30,
    "arguments_dimensions": 10,
    "d_min": -10,
    "d_max": 10
}"""

### Funkcja Griewank
Przeprowadzono doświadczenie dla dwóch zestawów parametrów wagi inercyjnej, składników kognitywnego oraz socjalnego.

<!-- <table style="width:150px;font-size:20px;text-align:left;" >
  <tr style="text-align:left;">
    <th>W</th>
    <th>C1</th> 
    <th>C2</th>
  </tr>
  <tr>
    <td>0.5</td>
    <td>0.8</td>
    <td>0.8</td>
  </tr>
  <tr>
    <td>0.5</td>
    <td>0.2</td>
    <td>0.4</td>
  </tr>
</table> -->

In [10]:
cfg = json.loads(json_config, object_hook = as_config)
for i in range(7):
    App().run("c1")
    cfg.c1 = cfg.c1-0.1

Fitness function value: 0.2638724548750093 achieved for c1 = 0.9.
The best solution is in: [ 9.4619773   4.66607399  0.32056301  5.96875723 -6.31257436  1.06951611
 -8.46089282  8.86652809  0.06362297  0.2368261 ].
Achieved in 2000 iterations
Fitness function value: 0.17378528759596024 achieved for c1 = 0.8.
The best solution is in: [ 0.0829191  -4.37216281 -0.08826428  0.57964876  0.4571807   7.32517983
 -0.14988825  0.14289051 -1.15870204  0.39931021].
Achieved in 2000 iterations
Fitness function value: 0.4188554780198983 achieved for c1 = 0.7000000000000001.
The best solution is in: [ 3.57734067  0.26771718  0.44395137 -0.4215207   0.88807791  0.01937893
  0.68623636 -8.00876944 -1.18915256  1.33421421].
Achieved in 2000 iterations
Fitness function value: 0.20489145155016064 achieved for c1 = 0.6000000000000001.
The best solution is in: [-0.11889734 -0.02631776 -0.01022821  6.13648033  1.18625915  0.21730889
 -0.30800365 -8.31313058  0.21697584  0.28005901].
Achieved in 2000 iterati

#### Przyjęto C1 = 0.8 dla którego uzyskano najlepsze wyniki.

In [16]:
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "griewank",
    "w": 0.5,
    "c1": 0.4,
    "c2": 0.9,
    "custom_c2": false,
    "target_error": 0,
    "iterations": 2000,
    "n_particles": 30,
    "arguments_dimensions": 10,
    "d_min": -10,
    "d_max": 10
}"""

In [17]:
cfg = json.loads(json_config, object_hook = as_config)
for i in range(7):
    App().run("c2")
    cfg.c2 = cfg.c2-0.1

Fitness function value: 0.2076220024986547 achieved for c2 = 0.9.
The best solution is in: [ 6.28726039 -4.3304849  -5.54027283 -6.2034371   7.13604201 -0.3049966
  8.05706645 -9.04268148  1.03488539 -1.01994789].
Achieved in 2000 iterations
Fitness function value: 0.2723980650279414 achieved for c2 = 0.8.
The best solution is in: [ 6.23727911  8.8991006  10.          0.0166595  -0.03513246 -8.02049168
  0.16992922  8.21140317 -0.36274667 -0.63340828].
Achieved in 2000 iterations
Fitness function value: 0.5697421528000062 achieved for c2 = 0.7000000000000001.
The best solution is in: [ 0.10282422  0.54201561 -4.99183411  0.64469324 -6.51305603  7.72335526
 -0.7512729  -0.45410036 -6.78315441  0.96243239].
Achieved in 2000 iterations
Fitness function value: 0.22946197970067128 achieved for c2 = 0.6000000000000001.
The best solution is in: [-3.19859065 -4.33906133  5.22098431 -5.92134977 -0.39723121 -0.65246393
  7.76173975 -8.03275479 -0.5174859   0.75636885].
Achieved in 2000 iteration

#### Przyjęto C2 = 0.9, ponieważ poniżej 0.9 zaobserwowano spadek jakości rozwiązania

In [18]:
cfg = json.loads(json_config, object_hook = as_config)
cfg.c1 = 0.8
cfg.c1 = 0.9
cfg.w = 0.9
for i in range(7):
    App().run("w")
    cfg.w = cfg.w-0.1

Fitness function value: 0.08616345492036392 achieved for w = 0.9.
The best solution is in: [ 9.42001139e+00  9.45770578e-06  5.43324041e+00 -6.27060594e+00
 -7.00731482e+00  7.67234003e+00  1.10792807e-04 -8.85031142e+00
 -4.49690800e-05  1.69975511e-04].
Achieved in 2000 iterations
Fitness function value: 0.07155887299664565 achieved for w = 0.8.
The best solution is in: [-3.13731034e+00  4.43807945e+00 -5.42912489e+00  1.91591231e-02
  1.69825685e-03  7.63078248e+00 -8.30521493e+00  1.38807550e-02
 -5.10363162e-03 -9.87763390e+00].
Achieved in 2000 iterations
Fitness function value: 0.053506163367269766 achieved for w = 0.7000000000000001.
The best solution is in: [ 3.15999635 -0.01485948 -5.3981026   0.08397339 -7.07333697  0.17769714
 -0.16826394 -0.20633602  9.3395724  -0.03110578].
Achieved in 2000 iterations
Fitness function value: 0.08900632870910063 achieved for w = 0.6000000000000001.
The best solution is in: [-3.11244177  8.94812526 -5.45353155 -0.05042419 -0.05225582 -0.061

#### Przyjęto w = 0.5 dla którego uzyskano najlepsze wyniki.

### Funkcja *zakharov*
Zaimplementowano możliwość zastosowania malejącej liniowo wraz z rosnącym numerem iteracji wartości składnika socjalnego. Użycie tej modyfikacji kontrolowane jest poprzez wartość klucza "custom_c2", odpowiednio na "true" lub "false".  
Zastosowanie tego rozwiązania przetestowano na przykładzie funkcji "Zakharov" dla przedstawionych poniżej parametrów.  
W przeprowadzonym eksperymencie, zastosowanie tej modyfikacji umożliwiło osiągnięcie poprawy wyniku z **12.62 do 10.82**
<table style="width:1000px;font-size:20px;text-align:left;" >
  <tr style="text-align:left;">
    <th>Funkcja</th>
    <th>mss</th>
    <th>num_dim</th>
    <th>range</th>
    <th>W</th>
    <th>C1</th> 
    <th>C2</th>
  </tr>
  <tr style="text-align:left;">
    <th>Zakharov</th>
    <th>Nie</th>
      <th>20</th>
      <th>&lt;-10, 10&gt; </th>
      <th>0.5</th>
      <th>0.8</th>
      <th>0.9</th>
  </tr>
  <tr style="text-align:left;">
    <th>Zakharov</th>
    <th>Tak</th>
      <th>20</th>
      <th>&lt;-10, 10&gt; </th>
      <th>0.5</th>
      <th>0.8</th>
      <th>0.9</th>
  </tr>
</table>
<div>
    <p>mss - Malejący składnik socjalny</p>
    <p>num_dim - Liczba wymiarów przestrzeni poszukiwań</p>
    <p>range - Przedział wartości pojedynczego wymiaru</p>
</div>

In [12]:
seed = 1
from numpy import random as nrand
nrand.seed(seed)
from random import seed
seed(seed)
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "zakharov",
    "w": 0.5,
    "c1": 0.8,
    "c2": 0.9,
    "custom_c2": false,
    "target_error": 0,
    "iterations": 2000,
    "n_particles": 50,
    "arguments_dimensions": 20,
    "d_min": -10,
    "d_max": 10
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

Fitness function value: 62.60010306394889 achieved for arguments_dimensions = 20.
The best solution is in: [ 0.21337415 -1.12064921  1.83092286  0.53837097 -3.20568751 -2.07894863
  2.02222887 -0.60953272 -0.03421801  1.11465259 -4.02704102  2.80106827
  1.48415861  0.45655883  0.95300907 -1.45710622  1.06102408  0.9309514
  0.45059682 -2.35788394].
Achieved in 2000 iterations


In [13]:
seed = 1
from numpy import random as nrand
nrand.seed(seed)
from random import seed
seed(seed)
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "zakharov",
    "w": 0.5,
    "c1": 0.8,
    "c2": 0.9,
    "custom_c2": true,
    "target_error": 0,
    "iterations": 2000,
    "n_particles": 50,
    "arguments_dimensions": 20,
    "d_min": -10,
    "d_max": 10
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

Fitness function value: 54.733619336857856 achieved for arguments_dimensions = 20.
The best solution is in: [ 1.08521838 -0.72979307  1.89888906 -1.59426515 -3.3178446  -1.96833027
  0.30231872  0.16782884 -0.4556276   0.19012749 -2.55732463  3.14843195
 -0.95991309  0.40811325  2.20686811 -0.25990484  0.71530134 -1.23666
  1.62410106 -1.102509  ].
Achieved in 2000 iterations


<h4>Korzystając z malejącej wartości składnika socjalnego udało się w badanym przypadku uzyskać poprawę z 62.60 do 47.17.</h4>

## "Particle Swarm Optimization" vs "Genetic Algorithm"
Zastosowano algorytmy: PSO oraz GA dla funkcji oznaczonej "f2" dla następujących parametrów:
<table style="width:1000px;font-size:20px;text-align:left;" >
  <tr style="text-align:left;">
    <th>Algorytm</th>
    <th>Funkcja</th>
    <th>Wynik</th>
    <th>num_dim</th>
    <th>range</th>
    <th>particles</th>
    <th>W</th>
    <th>C1</th> 
    <th>C2</th>
  </tr>
  <tr style="text-align:left;">
    <th>PSO</th>
    <th>F2</th>
    <th>863.68</th>
    <th>20</th>
    <th>&lt;-10, 10&gt; </th>
    <th>50</th>
    <th>0.5</th>
    <th>0.8</th>
    <th>0.9</th>
  </tr>
  <tr style="text-align:left;">
    <th>GA</th>
    <th>F2</th>
    <th>899.17</th>
    <th>20</th>
    <th>&lt;-10, 10&gt; </th>
    <th>50</th>
    <th>0.5</th>
    <th>0.8</th>
    <th>0.9</th>
  </tr>
  <tr style="text-align:left;">
    <th>PSO</th>
    <th>Griewank</th>
    <th>26.07</th>
    <th>20</th>
    <th>&lt;-600, 600&gt; </th>
    <th>50</th>
    <th>0.5</th>
    <th>0.8</th>
    <th>0.9</th>
  </tr>
  <tr style="text-align:left;">
    <th>GA</th>
    <th>Griewank</th>
    <th>0</th>
    <th>20</th>
    <th>&lt;-600, 600&gt; </th>
    <th>50</th>
    <th>0.5</th>
    <th>0.8</th>
    <th>0.9</th>
  </tr>
</table>
<p>num_dim - Liczba wymiarów przestrzeni poszukiwań
<p>range - Przedział wartości pojedynczego wymiaru</p>

### Funkcja "F2"

In [200]:
seed = 1
from numpy import random as nrand
nrand.seed(seed)
from random import seed
seed(seed)
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "f2",
    "w": 0.5,
    "c1": 0.8,
    "c2": 0.9,
    "target_error": 1e-8,
    "iterations": 2000,
    "n_particles": 50,
    "arguments_dimensions": 20,
    "d_min": -10,
    "d_max": 10
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

Fitness function value: 863.6844722893944 achieved for arguments_dimensions = 20.
The best solution is in: [ 10.           3.05117549  10.          10.         -10.
  10.           2.96169799  10.          10.          -0.3492853
  10.          -0.48201797  10.          10.          10.
  10.          10.          10.          10.          10.        ].
Achieved in 375 iterations


In [201]:
seed = 1
from numpy import random as nrand
nrand.seed(seed)
from random import seed
seed(seed)
json_config ="""
{
    "algorithmType": "ga",
    "fitness": "f2",
    "w": 0.5,
    "c1": 0.8,
    "c2": 0.9,
    "target_error": 0,
    "iterations": 2000,
    "n_particles": 50,
    "arguments_dimensions": 20,
    "d_min": -10,
    "d_max": 10
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

{'gen': 2000,
 'nevals': 23,
 'avg': 902.6260403509935,
 'std': 21.414787742460934,
 'min': 899.1747972837709,
 'max': 1051.7369506449022}

### Funkcja "Griewank"

In [202]:
seed = 0
from numpy import random as nrand
nrand.seed(seed)
from random import seed
seed(seed)
json_config ="""
{
    "algorithmType": "pso",
    "fitness": "griewank",
    "w": 0.5,
    "c1": 0.8,
    "c2": 0.9,
    "target_error": 1e-8,
    "iterations": 2000,
    "n_particles": 50,
    "arguments_dimensions": 20,
    "d_min": -600,
    "d_max": 600
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

Fitness function value: 26.071892593564776 achieved for arguments_dimensions = 20.
The best solution is in: [  84.33067941    6.05841196  -87.4386228    70.15562891   55.00432473
   29.71836905    3.59683962  150.95213716   13.79839486  -12.45236875
   17.12842152  -67.08823622  -59.93865732  135.42280788  -10.27577556
    9.95351664  -17.11382024 -108.92291948    6.03422949  119.97466975].
Achieved in 8 iterations


In [203]:
seed = 1
from numpy import random as nrand
nrand.seed(seed)
from random import seed
seed(seed)
json_config ="""
{
    "algorithmType": "ga",
    "fitness": "griewank",
    "w": 0.5,
    "c1": 0.8,
    "c2": 0.9,
    "target_error": 0.1,
    "iterations": 2000,
    "n_particles": 50,
    "arguments_dimensions": 20,
    "d_min": -600,
    "d_max": 600
}"""
cfg = json.loads(json_config, object_hook = as_config)
App().run()

{'gen': 2000,
 'nevals': 23,
 'avg': 0.004274253959698218,
 'std': 0.019219458920622365,
 'min': 0.0,
 'max': 0.12266743810962721}

<h1><center>Wnioski</center></h1>
<p>
<ul style="text-align: justify;text-justify: inter-word">
    <li>Znalezienie optymalnych wartości parametrów konfiguracyjnych współczynników kognitywnego, 
    socjalnego oraz redukcji prędkości ma drastyczny wpływ na jakość rozwiązania. Dlatego też, powinna być przeprowadzana staranna analiza mająca na celu dobór odpowiednich wartości
    do konkretnego rozwiązywanego problemu. Jednocześnie, heurystyczne metody pozwalające oszacować wstępnie odpowiednie wartości mogą okazać się bardzo przydatne.</li>
    <li>Dla funkcji F2 oraz Griewank lepsze efekty osiągnięto z wykorzystaniem odpowiednio algorytmu PSO oraz genetycznego.  Zatem dobór odpowiedniej metody optymalizacji również powinien być przeprowadzany z uwzględnieniem rozwiązywanego problemu.</li>
    <li>Z wykorzystaniem metody malejącej wartości składnika socjalnego udało się uzyskać istotną poprawę wyniku dla badanego problemu.</li>
</ul>
</p>